In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import xarray as xr

fs=20						#fontsize for labels, legend, ...
font={'family' : 'serif', 'size' : fs}		#define font for legend
plt.rc('font', **font)	

In [ ]:
def area_mean(x):
    weights=np.cos(lats*np.pi/180)
    return sum(weights*x)/sum(weights)

# EBM Without Ice

In [ ]:
lats_star=np.linspace(-90, 90, num=181)
x_star=np.sin(lats_star*np.pi/180)
lats=(lats_star[:-1]+ lats_star[1:])/2
x=np.sin(lats*np.pi/180.)
dx=np.diff(x)
dx_star=np.diff(x_star)

In [ ]:
A=-388.74   #W/m^2 for T in Kelvin (note: A_degC = 210 (in Brian Rose Lecture) =? 273.15 * B + A_K)
B=2.17      #W/m^2 /K
C=4e7       #J/K for either whole atmosphere or 10m water
D=0.55      #Diffusion parameter = m^2/s getuned
Q=1361./4*(1-0.477*0.5*(3*x**2-1))  #solar insolation (lats) in W/m^2
alpha=0.3   #constant albedo TODO: change to non-constant
print("This should reproduce S0: ",area_mean(Q*4))

In [ ]:
def calc_diff(T):
    F=-D*np.diff(T)/dx 
    F=np.insert(F,0,0.) #Boundary_condition South Pole F=0
    F=np.append(F,0.)   #Boundary condition North Pole F=0
    diffusion=-np.diff(F * (1-x_star**2))/dx_star
    return diffusion, F

In [ ]:
def get_P_hum_distribution(P_glob=0.034):
    a=0.16180316; b= 39.67767011; c=16.63696653
    dist= a*np.exp(-(lats-b)**2/c**2)
    fit_glob=area_mean(dist)
    fit_normalized=P_glob/fit_glob*dist
    test_P_glob=area_mean(fit_normalized)
    return fit_normalized
P_hum_dist=get_P_hum_distribution(P_glob=0.34)

In [ ]:
P_hum0034=get_P_hum_distribution(P_glob=0.034)
nr_years=15.
dt=3600.*1. #s 
N=int(nr_years*365*3600*24/dt)
T=np.array([288 for item in lats])
T_D0=np.array([288 for item in lats])

for i in range(0,N):
    diffusion, F= calc_diff(T)
    dTdt=Q*(1-alpha) - (A+B*T) + diffusion + P_hum0034
    dTdt_noD=Q*(1-alpha) - (A+B*T_D0) + P_hum0034
    T=T+ dt/C * (dTdt)
    T_D0=T_D0+dt/C* (dTdt_noD)
    

In [ ]:
P_hum034=get_P_hum_distribution(P_glob=0.34)

T_P=np.array([288 for item in lats])
T_P_D0=np.array([288 for item in lats])
for i in range(0,N):
    diffusion, F= calc_diff(T_P)
    dTdt=Q*(1-alpha) - (A+B*T_P) + diffusion  + P_hum034
    dTdt_D0=Q*(1-alpha) - (A+B*T_P_D0)  + P_hum034
    T_P = T_P+ dt/C * (dTdt)
    T_P_D0 = T_P_D0+ dt/C * (dTdt_D0)

plt.plot(lats, T_P-T, '-o', label='No Ice, D=0.55')
plt.plot(lats, T_P_D0-T_D0, '-x', label='No Ice, D=0')
print(area_mean(T_P-T))

# WITH ICE
Import the simulated data, actually this is for t=116 and hence P=0.338... but whatever

In [ ]:
#data=np.loadtxt("/home/peter/PIK/Woche KW4/EBM_to_cluster/15yr_3/"+"data_T_time_Tf10_georg_T.txt")
#data1=np.loadtxt("/home/peter/PIK/Woche KW4/EBM_to_cluster/15yr_Boltz/"+"data_T_time_Tf10_georg_T.txt")
#data2=np.loadtxt("/home/peter/PIK/Woche KW4/EBM_to_cluster/15yr_smallD_gaussP/"+"data_T_time_Tf10_georg_T.txt")
#data2=np.loadtxt("/home/peter/PIK/Woche KW4/EBM_to_cluster/15yr_smallDTUNED/"+"data_T_time_Tf10_dyn_georg_T.txt")
#data3=np.loadtxt("/home/peter/PIK/Woche KW4/EBM_to_cluster/15yr_Boltz_smallDTUNED/"+"data_T_time_Tf10_dyn_georg_T.txt")
##NOTE: SMALL D GAUSS IS NOT GAUSS????#
#
#T_034_10_georg=data[116,:]
#T_0034_10_georg=data[0,:]
#
#T_034_10_boltz=data1[116,:]
#T_0034_10_boltz=data1[0,:]
#
#T_034_10_smallD_G=data2[116,:]
#T_0034_10_smallD_G=data2[0,:]
#
#T_034_10_smallD_boltz=data3[116,:]
#T_0034_10_smallD_boltz=data3[0,:]

In [ ]:
folder="/home/peter/PIK/EBM_equilibrium/BsD-10G/"
data=np.loadtxt(folder+"BsD-10G_T_over_theta0.034.txt")
T_0034_10_smallD_boltz=data[:]
data=np.loadtxt(folder+"BsD-10G_T_over_theta0.34.txt")
T_034_10_smallD_boltz=data[:]

folder="/home/peter/PIK/EBM_equilibrium/LMsD-10G/"
data=np.loadtxt(folder+"LMsD-10G_T_over_theta0.034.txt")
T_0034_10_smallD_G=data[:]
data=np.loadtxt(folder+"LMsD-10G_T_over_theta0.34.txt")
T_034_10_smallD_G=data[:]

folder="/home/peter/PIK/EBM_equilibrium/LMD-10G/"
data=np.loadtxt(folder+"LMD-10G_T_over_theta0.034.txt")
T_0034_10_LM=data[:]
data=np.loadtxt(folder+"LMD-10G_T_over_theta0.34.txt")
T_034_10_LM=data[:]

folder="/home/peter/PIK/EBM_equilibrium/BD-10G/"
data=np.loadtxt(folder+"BD-10G_T_over_theta0.034.txt")
T_0034_10_boltz=data[:]
data=np.loadtxt(folder+"BD-10G_T_over_theta0.34.txt")
T_034_10_boltz=data[:]

#folder="/home/peter/PIK/EBM_equilibrium/LMsD-2G/"
#data=np.loadtxt(folder+"LMsD-2G_T_over_theta0.034.txt")
#T_0034_2_smallD_G=data[:]
#data=np.loadtxt(folder+"LMsD-2G_T_over_theta0.34.txt")
#T_034_2_smallD_G=data[:]

In [ ]:
fig=plt.figure(figsize=(16,9))
ax=fig.add_subplot(111)
ax.plot(lats,T_034_10_LM-T_0034_10_LM, '-', lw=4, color='red', 
        label='$LMD_{-10}$ (with $D=0.418$)')
#        label='With ice for $T<T_f=-10^\circ C$, (Mcguffie and Hendersson-Sellers, 2014), $D=0.42$')
ax.plot(lats,T_034_10_smallD_G-T_0034_10_smallD_G, '-', lw=4, color='orange', 
        label='$LMsD_{-10}$ (with $D=0.395$)')
#        label='With ice for $T<T_f=-10^\circ C$, (Mcguffie and Hendersson-Sellers, 2014), $D=0.39$')
#ax.plot(lats,T_034_2_smallD_G-T_0034_2_smallD_G, '-', lw=4, color='orange', 
#        label='$LMsD_{-2}$ (with $D=0.792$)')
##        label='With ice for $T<T_f=-10^\circ C$, (Mcguffie and Hendersson-Sellers, 2014), $D=0.39$')
ax.plot(lats,T_034_10_boltz-T_0034_10_boltz, '-', lw=4, color='darkgreen',
        label='$BD_{-10}$ (with $D=0.316$)')
#        label='With ice for $T<T_f=-10^\circ C$, Boltzmann, $D=0.32$')
#ax.plot(lats,T_034_10_smallD_M-T_0034_10_smallD_M, '-', lw=4, color='orange', label='Ice at $T_f=-10^\circ C$, LM, $D=0.34$')

ax.plot(lats,T_034_10_smallD_boltz-T_0034_10_smallD_boltz, '-', lw=4, color='lime', 
        label='$BsD_{-10}$ (with $D=0.281$)')
ax.plot(lats, T_P-T, '--', color='darkblue', lw=3, label='EBM-1D no ice, $D=0.55$')
ax.plot(lats, T_P_D0-T_D0, '--',color='c', lw=3, label='EBM-1D no ice, $D=0$')
ax.set_xlabel(r"Latitudes $\theta$ in $^\circ$")
ax.set_xticks(np.arange(-90, 91, step=30))
ax.set_xlim(-90,90)
ax.set_ylabel(r"$ \Delta T (\theta)  [{\rm K}]$")
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")

left, bottom, width, height = [0.45, 0.5, 0.25, 0.25]
ax2 = fig.add_axes([left, bottom, width, height])
ax2.set_yticks(ax2.get_yticks(), minor=True)
#ax2.grid(b=True, which='minor')
ax2.set_xticks([-90,0,90])
#ax2.axis.xticks.params(labelsize=12)
ax2.plot(lats, T_034_10_LM, '-r', label=r'$P=0.34 \, {\rm W/m^2}$')
ax2.plot(lats, T_0034_10_LM, ':r', label=r'$P=0.034\, {\rm W/m^2}$')
ax2.minorticks_on()
ax2.grid(b=True, which='major')
ax2.grid(b=True, which='minor', alpha=0.4)
ax2.set_ylabel(r"$T\, [{\rm K}]$", rotation=360)
ax2.yaxis.set_label_coords(-0.025, 1.025)
ax2.legend(fontsize=17, loc=(0.65, 0.85))
ax2.tick_params(axis='both', labelsize=19)
ax.grid()
ax.legend()
ax.set_title(r"Temperature change over latitudes due to $\langle P_{\rm hum}\rangle_\theta=0.34\, {\rm W/m^2}$ (${\rm G_F}$)"+'\n'+"for different EBM-1D model settings")
print(area_mean(T_034_10_LM-T_0034_10_LM))
#plt.tight_layout()
ax.arrow(42,1.58, 63-(42),1.54-1.54 , head_width=0.1, head_length=2,color='red')
plt.savefig("delT(theta)_Tf-10_different_models.png")

In [ ]:
fig=plt.figure(figsize=(16,9))
ax=fig.add_subplot(111)

ax.plot(lats,T_034_10_boltz-T_0034_10_boltz, '-', lw=4, color='green',
        label='larger $D$ (thesis: $BD_{-10}$)')
ax.plot(lats,T_034_10_smallD_boltz-T_0034_10_smallD_boltz, '-', lw=4, color='blue', 
        label='smaller $D$ (thesis: $BsD_{-10}$), no ice loss in this case')
ax.set_xlabel(r"Latitudes $\theta$ in $^\circ$")
ax.set_xticks(np.arange(-90, 91, step=30))
ax.set_xlim(-90,90)
ax.set_ylabel("$\Delta T (\phi)  [K]$")
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")

left, bottom, width, height = [0.1, 0.35, 0.25, 0.25]
ax2 = fig.add_axes([left, bottom, width, height])
ax2.set_yticks(ax2.get_yticks(), minor=True)
#ax2.grid(b=True, which='minor')
ax2.set_xticks([-90,0,90])
#ax2.axis.xticks.params(labelsize=12)
ax2.plot(lats, T_034_10_boltz, '-', color='green', label='$P=0.34 \, W/m^2$')
ax2.plot(lats, T_0034_10_boltz, ':', color='green', label='$P=0.034\, W/m^2$')
ax2.minorticks_on()
ax2.grid(b=True, which='major')
ax2.grid(b=True, which='minor', alpha=0.4)
ax2.set_ylabel("$T\, [K]$", rotation=360)
ax2.yaxis.set_label_coords(-0.025, 1.025)
ax2.tick_params(axis='both', labelsize=19)
ax2.legend(fontsize=17, loc=(0.65, 0.85))
ax.grid()
ax.legend()
ax.set_title("Temperature change over latitudes due to $P_{hum}=0.34\, W/m^2$ ($G_F$)")#+'\n'+"for different EBM-1D model settings")
print(area_mean(T_034_10_LM-T_0034_10_LM))
plt.tight_layout()
ax.arrow(-27,0.7, 39-(-27),0.5-0.7 , head_width=0.05, head_length=2,color='green')
plt.savefig("delT(theta)_Tf-10_different_models_Presentation.png")

# Plot Temperature profile over observed data from NCEP (see Brian Rose Lecture)


In [ ]:
#  daily surface temperature from  NCEP reanalysis
#(see Lecture 14 in BRIAN ROSE)
import xarray as xr
ncep_url = "http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/"
ncep_temp = xr.open_dataset( ncep_url + "surface_gauss/skt.sfc.day.1981-2010.ltm.nc", decode_times=False)
ncep_temp_zon = ncep_temp.skt.mean(dim='lon')
#ncep_temp_zon.mean(dim='time')

In [ ]:
#fig=plt.figure(figsize=(8,5))
#ax=fig.add_subplot(111)
#ax.plot(ncep_temp_zon.lat, ncep_temp_zon.mean(dim='time'), label='NCEP Daily Temp '+
#         '\n'+'(latitudinal mean)'+'\n'+'avg. from 1981-2010', lw=3)
#ax.plot(lats, T_0034_10_georg, '-.r', label='(LG),     $D=0.42$, $T_f=-10^\circ C$', lw=2)
#ax.plot(lats, T_0034_10_smallD_G, '--y', label='(LGsD), $D=0.34$, $T_f=-10^\circ C$', lw=2)
#ax.set_xlabel("Latitudes in degrees")
#ax.set_xticks([-90,0,90])
#ax.set_xlim(-90,90)
#ax.tick_params(axis = 'x', which = 'minor', labelsize = 0)
#ax.set_xticks([-60,-30,30,60], minor=True)
#ax.minorticks_on()
#ax.set_ylabel("$T(\phi)\, [K]$")
#ax.grid(True, which='both')
#ax.legend(fontsize=15)
#fig.tight_layout()
#plt.savefig("T(phi)_Ncep_differentD.png")


# Reproduce this average from ncep!

"http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/"
https://www.esrl.noaa.gov/psd/thredds/catalog/Datasets/ncep.reanalysis2.dailyavgs/gaussian_grid/catalog.html?dataset=Datasets/ncep.reanalysis2.dailyavgs/gaussian_grid/skt.sfc.gauss.1979.nc
ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis2.dailyavgs/gaussian_grid/skt.sfc.gauss.1979.nc

In [ ]:
#url="https://www.esrl.noaa.gov/psd/thredds/catalog/Datasets/ncep.reanalysis2.dailyavgs/gaussian_grid/catalog.html?dataset=Datasets/ncep.reanalysis2.dailyavgs/gaussian_grid/skt.sfc.gauss.1979.nc"
url="https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis2.dailyavgs/gaussian_grid/skt.sfc.gauss.1988.nc"
data=xr.open_dataset(url, decode_times=False,decode_cf=False)

In [ ]:
#data2=data.mean(dim='lon')
#data3=data2.mean(dim='time')
data4=data.mean(dim=('lon', 'time'))

In [ ]:
#unpacked value = add_offset + ( (packed value) * scale_factor ) 
temp=data4.skt*data.skt.scale_factor + data.skt.add_offset

#### Air 2m temperature and compare with skt

In [ ]:
url4="https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis2/gaussian_grid/air.2m.gauss.1988.nc"
#/psd/thredds/dodsC/Datasets/ncep.reanalysis2/gaussian_grid/air.2m.gauss.1979.nc
data2m=xr.open_dataset(url4, decode_times=False,decode_cf=False)
data2m2=data2m.mean(dim='time')
data2m3=data2m2.mean(dim='lon')

In [ ]:
temp2=data2m3.air[0]*data2m.air.scale_factor + data2m.air.add_offset
plt.plot(data2m3.lat, temp2)#data2m3.air[0])
plt.plot(data4.lat, temp)#data2m3.air[0])

### automatise (DONE ONCE,  now load from FILE)

In [ ]:
#years=np.arange(1979, 2018, step=1)
#print(years)
#filler=np.zeros([len(years), len(data.lat.values)])
#T=xr.DataArray(filler,
#               coords={'time': years, 'lat':data.lat}, #data.lat.values
#               dims=('time', 'lat'))
#url_pre="https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis2.dailyavgs/gaussian_grid/"
#for year in years:
#    print(str(year)+', ', end='')
#    url="skt.sfc.gauss."+str(year)+".nc"
#    data=xr.open_dataset(url_pre+url, decode_times=False,decode_cf=False)
#    mean_data=data.skt.mean(dim=('lon', 'time'))
#    mean_T=mean_data*data.skt.scale_factor + data.skt.add_offset
#    T.sel(time=year)[:]=mean_T
#T.to_netcdf("Tskt(lat)_1979-2017_ncep.nc")

In [ ]:
T=xr.open_dataset("Tskt(lat)_1979-2017_ncep.nc")
plt.figure(figsize=(8,5))
for year in T.time:
    plt.plot(T.lat, T.sel(time=year).__xarray_dataarray_variable__, ':r', alpha=0.4)
#plt.plot(ncep_temp_zon.lat, ncep_temp_zon.mean(dim='time'), '--g', label='NCEP Daily Temp '+
#         '\n'+'(latitudinal mean)'+'\n'+'avg. from 1981-2010', lw=2)
meanT=T.__xarray_dataarray_variable__.mean(dim='time')
#plt.plot(T.lat, meanT, '-b', lw=3 )


In [ ]:
fig=plt.figure(figsize=(16,5))
ax=fig.add_subplot(111)
ax.plot(T.lat, meanT, '-b', label='NCEP-DOE Reanalysis 2'+
        '\n'+r"Daily $T(\theta)$"+'\n'+'averaged from 1979-2017', lw=3)
ax.plot(lats, T_0034_10_LM, '-.r', label='(LMD),  $D=0.418$, $T_f=-10^\circ {\rm C}$', lw=2)
ax.plot(lats, T_0034_10_smallD_G, '--y', label='(LMsD), $D=0.395$, $T_f=-10^\circ {\rm C}$', lw=2)
ax.set_xlabel(r"Latitudes $\theta$ in $^\circ$")
ax.set_xticks([-90,0,90])
ax.set_xlim(-90,90)
ax.tick_params(axis = 'both', which = 'major', labelsize = 18)
ax.tick_params(axis = 'x', which = 'minor', labelsize = 0)
ax.set_xticks([-60,-30,30,60], minor=True)
ax.minorticks_on()
ax.set_ylabel(r"$T(\theta)\, [{\rm K}]$")
ax.grid(True, which='both')
ax.legend(fontsize=18)
fig.tight_layout()
plt.savefig("T(theta)_Ncep_differentD.png")